Funcoes utilizadas
Criar uma lista de comprimentos de strings em cada célula da coluna especificada
comprimentos = [len(str(celula)) for celula in df.iloc[: ,4]]
Encontrar o índice da célula com o maior número de caracteres
indice_maior = comprimentos.index(max(comprimentos))
Extrair a célula com o maior número de caracteres
celula_maior = df.iloc[indice_maior][4] df.iloc[: , 4][indice_maior] df.iloc[: , 4].value_counts() celula_maior df.iloc[:,0]
index = df.columns.get_loc(‘abcz’)
index df

IDANIMAL = CHAVE UNICA
MP = CHAVE UNICA -  Mes parto
AP = CHAVE UNICA - -Ano parto

LTOT = lactacao total
L365 = lactacao em ate 365 dias
L305 = lactacao em ate 305 dias
DE = Data de Secagem
ME = Mes de secagem
AE = ANO de Secagem
caso o LTOT, L365 e L305, DE, ME e AE sejam iguais ao arquivo old, pode ser descartado

In [1]:
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

## conectando no banco de dados

In [2]:
import mysql.connector

mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

## Retornando toda tabela genealogia_embrapa e genealogia_abcz e tabela criadores do banco de dados para dataframes

In [3]:
sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
sql1 = "SELECT * FROM genealogia_abcz"
sql2 = "SELECT * FROM criadores"
sql3 = "SELECT * FROM producao_embrapa"
df_genealogia_embrapa = pd.read_sql(sql, mydb)
df_genealogia_embrapa=df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = pd.read_sql(sql1, mydb)
df_criadores = pd.read_sql(sql2, mydb)
df_producao = pd.read_sql(sql3, mydb)


## Lendo planilhas

In [4]:
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [ ]:
#pedigree

In [ ]:
#dados_2022

In [ ]:
#dados_2023

In [ ]:
#df_genealogia_embrapa

In [ ]:
df_genealogia_abcz

In [ ]:
#df_criadores

In [ ]:
#df_producao

## Criando um dataframe que é a junção dos dois dataframes, dados_2022 e dados_2023, pelas celulas que possuem os mesmos valores nas colunas IDANIMAL, AP, MP, L305, LTOT, L365, ME e AE.

In [5]:
df_dados_sem_atualizacao_entre_2022_2023 = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop(df_dados_sem_atualizacao_entre_2022_2023.filter(regex='_drop$').columns, axis=1)
df_dados_sem_atualizacao_entre_2022_2023 = df_dados_sem_atualizacao_entre_2022_2023.drop_duplicates()
#df_dados_sem_atualizacao_entre_2022_2023

## Criando um dataframe com somente os novos dados inseridos em 2023
Esses são os dados que precisam ser inseridos no banco de dados

In [6]:
# faz o merge entre os dois dataframes, o dados com os dados repetidos
df_merge = pd.merge(dados_2023, df_dados_sem_atualizacao_entre_2022_2023, how='left', indicator=True, suffixes=('', '_drop'))

# filtra somente as linhas que estão no dataframe da esquerda
df_dados_novos_para_insercao_no_banco_de_dados = df_merge[df_merge['_merge'] == 'left_only']

# remove a coluna "_merge"
df_dados_novos_para_insercao_no_banco_de_dados.drop('_merge', axis=1, inplace=True)

df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_merge.filter(regex='_drop$').columns, axis=1)
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados

# Adicionando coluna GS, oriunda do dataframe Pedigree, no dataframe df_dados_novos_para_insercao_no_banco_de_dados
inclusão necessária para localizar o animal

In [7]:
df_temporaria = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria
df_dados_novos_para_insercao_no_banco_de_dados

nome_antigo = 'SEXO'
nome_novo = 'sexo'
indice_coluna = df_dados_novos_para_insercao_no_banco_de_dados.columns.get_loc(nome_antigo)
df_dados_novos_para_insercao_no_banco_de_dados.rename(columns={nome_antigo: nome_novo}, inplace=True)
#df_dados_novos_para_insercao_no_banco_de_dados

# Criando subtabela com somente as linhas que possuem gs = 0 no dataframe df_dados_novos_para_insercao_no_banco_de_dados

In [8]:
sub_tabela_gs_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['gs'] == 0]
sub_tabela_gs_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,secagem,anopc,anop_menos1c,rebc,rac,nordc,rebanoe,ordem,gs,sexo
53,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,2021-04-14,2020,2019,5191,2,2,51912020222,NaN,0,F
79,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,2022-08-25,2022,2021,1945,2,2,19452022222,NaN,0,F
81,4667,DAMA F.INDIANA,11,SNFP140,FARDO FIV F.MUTUM,11,MUT697,SOBERANA F. INDIANA,11,Z3236,...,2022-01-28,2021,2020,4667,2,2,46672021222,NaN,0,F
99,4667,MEIA NOITE F.INDIANA,11,SNFN48,MAESTRO TE F.MUTUM,11,MUT214,ESTRELA F.INDIANA,11,SNFP87,...,2022-07-06,2021,2020,4667,2,2,46672020122,NaN,0,F
184,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,2022-07-13,2021,2020,5192,3,2,51922021232,NaN,0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,5192,PILLAR FIV DE BRAS.,11,RRP8130,DIAMANTE TE DE BRASILIA,11,RRP5640,JAMARIA FIV DE BRASILIA,11,RRP6958,...,2022-11-12,2022,2021,5192,3,2,51922021132,NaN,0,F
995,4667,IRLANDA F. INDIANA,11,SNFN16,FARDO FIV F.MUTUM,11,MUT697,LONDRINA F. INDIANA,11,SNFP44,...,2022-10-08,2022,2021,4667,2,2,46672022222,NaN,0,F
1000,6915,DEJUA DAS ARABIAS,11,RHBR63,MAJOR TE DOS POCOES,11,APPG801,DEJUA TE F.MUTUM,11,MUT395,...,2022-04-26,2021,2020,6915,2,2,69152021222,NaN,0,F
1002,6915,PRATA DAS ARABIAS,11,RHBR49,VALE OURO TE SILVANIA,11,EFC464,JADE,11,RHBR4,...,2022-03-01,2021,2020,6915,2,2,69152021222,NaN,0,F


# Adicionando coluna cod_prod, cod_pai e cod_mae, oriundos do dataframe df_genealogia_abcz, na subtabela gs_0 dos animais que sao gs=0

In [9]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
df_temporaria1 = pd.merge(sub_tabela_gs_0, df_genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
sub_tabela_gs_0=df_temporaria1
sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()
sub_tabela_gs_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae
0,5191,IBERICA FARDO,11,APKK18,FARDO FIV F.MUTUM,11,MUT697,ESPANHOLA,11,APKK1,...,5191,2,2,51912020222,NaN,0,F,13672891.0,8530023.0,10614116.0
1,1945,BROMELIA FIV DO JRD,11,JRDG28,RADAR DOS POCOES,11,A7368,PROFANA 6,11,EBQG1,...,1945,2,2,19452022222,NaN,0,F,12005290.0,1615985.0,10091814.0
2,4667,DAMA F.INDIANA,11,SNFP140,FARDO FIV F.MUTUM,11,MUT697,SOBERANA F. INDIANA,11,Z3236,...,4667,2,2,46672021222,NaN,0,F,12175694.0,8530023.0,10905624.0
3,4667,MEIA NOITE F.INDIANA,11,SNFN48,MAESTRO TE F.MUTUM,11,MUT214,ESTRELA F.INDIANA,11,SNFP87,...,4667,2,2,46672020122,NaN,0,F,12971292.0,5932826.0,9691642.0
4,5192,URUPIANGA FIV DE BRAS.,11,RRP8772,JAGUAR TE DO GAVIAO,11,GAV291,HONDA FIV DE BRAS.,11,RRP6268,...,5192,3,2,51922021232,NaN,0,F,17054525.0,4176523.0,13961623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,5192,PILLAR FIV DE BRAS.,11,RRP8130,DIAMANTE TE DE BRASILIA,11,RRP5640,JAMARIA FIV DE BRASILIA,11,RRP6958,...,5192,3,2,51922021132,NaN,0,F,14820082.0,6716196.0,11600828.0
75,4667,IRLANDA F. INDIANA,11,SNFN16,FARDO FIV F.MUTUM,11,MUT697,LONDRINA F. INDIANA,11,SNFP44,...,4667,2,2,46672022222,NaN,0,F,11921127.0,8530023.0,6352036.0
76,6915,DEJUA DAS ARABIAS,11,RHBR63,MAJOR TE DOS POCOES,11,APPG801,DEJUA TE F.MUTUM,11,MUT395,...,6915,2,2,69152021222,NaN,0,F,13478123.0,3196251.0,8006397.0
77,6915,PRATA DAS ARABIAS,11,RHBR49,VALE OURO TE SILVANIA,11,EFC464,JADE,11,RHBR4,...,6915,2,2,69152021222,NaN,0,F,11470928.0,3890526.0,9469537.0


# Adicionando codprod, cod_pai, cod_mae no dataframe df_dados_novos_para_insercao_no_banco_de_dados a partir do dataframe sub_tabela_gs_0

In [10]:
df_temporaria4 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados



,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,7461,2,2,74612019122,NaN,5,F,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,10639,2,2,106392021122,NaN,4,F,NaN,NaN,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,6039,2,2,60392020122,NaN,4,F,NaN,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,1945,2,2,19452021222,NaN,4,F,NaN,NaN,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,73119,2,2,731192021222,NaN,4,F,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,6029,2,2,60292021222,NaN,2,F,NaN,NaN,NaN
1039,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,132,2,2,1322019222,NaN,6,F,NaN,NaN,NaN
1040,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,6915,2,2,69152021222,NaN,5,F,NaN,NaN,NaN
1041,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,6029,2,2,60292019122,NaN,5,F,NaN,NaN,NaN


# Criando subtabela com os animais em que o gs da mae = 11

In [11]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0
# selecionar apenas as linhas que começam com letras
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando o cod_prod das maes no dataframe genealogia_abcz

In [12]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_mae_0


,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_mae_gen
0,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,NaN,9513273.0
1,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,NaN,10621711.0
2,83436,PURA FIV DOORMAN SJ LALU,12,2678-BK,VAL-BISSON DOORMAN,10,AX140558,KANETA DP,11,DPJ955,...,3,2,834362021232,NaN,4,F,NaN,NaN,NaN,12718455.0
3,2381,GOTA CHELIOS JM NOVO HORIZONTE,12,7006-BC,DOMICOLE CHELIOS,10,AX131313,DALIA NOVO HORIZONTE,11,JMCH76,...,2,2,23812021222,NaN,4,F,NaN,NaN,NaN,12866148.0
4,7461,NOIVA FIV HUNTER RF SAO FRANCI,12,5567-AK,COOKIECUTTER MOM HUNTER,10,AX137355,OFENSA DO U.A.,11,UAC2324,...,2,2,74612019122,NaN,4,F,NaN,NaN,NaN,8174644.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,11818,FABULOSA FIV BRADNICK DA JJPZ,12,3852-BN,REGANCREST-GV S BRADNICK- ET,10,AX134761,FABIA FIV,11,UDI823,...,3,2,118182021232,NaN,4,F,NaN,NaN,NaN,13327464.0
268,3304,JOIA FIV AFTERSHOCK DA MU-MU,12,9069-Y,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,LIBIA FIV RIO VALE,11,CAVE242,...,2,2,33042020122,NaN,4,F,NaN,NaN,NaN,12090148.0
269,2381,ESTIVA QUIZAR JM NOVO HORIZONT,12,9958-AS,COLLEM FEVER QUIZAR,10,AX134708,SC MANCHETE PALADINO,11,MJJS411,...,2,2,23812020122,NaN,4,F,NaN,NaN,NaN,4220695.0
270,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,2,19452022222,NaN,4,F,NaN,NaN,NaN,11428022.0


# Adicionando o cod_prod_mae_gen das maes no dataframe df_dados_novos

In [13]:
df_temporaria5 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_mae_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria4
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria5 = df_temporaria5.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria5
df_dados_novos_para_insercao_no_banco_de_dados


,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_mae_gen
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,NaN,9513273.0
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,NaN,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,NaN,10621711.0
5,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,2,60292021222,NaN,2,F,NaN,NaN,NaN,NaN
1117,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,NaN,NaN,NaN
1118,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,NaN,NaN,NaN
1119,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2,2,60292019122,NaN,5,F,NaN,NaN,NaN,NaN


# Criando subtabela em que o gs do pai = 11 e que os 3 primeiros caracteres do rgpai sao letras

In [14]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando o cod_prod dos pais no dataframe genealogia_abcz

In [15]:
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_pai_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_mae_gen,cod_prod_pai_gen
0,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,NaN,NaN,NaN,4320031.0
1,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,NaN,NaN,NaN,3540404.0
2,10586,JHV CANASTRA ENLEVO,12,5011-BG,ENLEVO SILVANIA,11,EFC717,JHV FORMATURA 315,10,C-3602,...,2,105862021132,NaN,4,F,NaN,NaN,NaN,NaN,6915068.0
3,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,60292021122,NaN,2,F,NaN,NaN,NaN,NaN,4176523.0
4,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,33042020122,NaN,4,F,NaN,NaN,NaN,NaN,3287666.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,9330,PALMADA CAL,14,0383-BN,DIORITO CAL,11,CAL9630,ILHOA FIV CAL,12,7229-AG,...,2,93302022232,NaN,2,F,NaN,NaN,NaN,NaN,11785662.0
453,5569,CAMPISTA MAJOR DO MORRO,14,3881-AT,MAJOR TE DOS POCOES,11,APPG801,SUELLEN II DO MORRO,12,2777-K,...,2,55692021222,NaN,2,F,NaN,NaN,NaN,NaN,3196251.0
454,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2,23812020122,NaN,2,F,NaN,NaN,NaN,NaN,3287666.0
455,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,2,55692020122,NaN,2,F,NaN,NaN,NaN,NaN,3890526.0


# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos

In [16]:
df_temporaria6 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria6 = df_temporaria6.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria6
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_mae_gen,cod_prod_pai_gen
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,74612019122,NaN,5,F,NaN,NaN,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,106392021122,NaN,4,F,NaN,NaN,NaN,9513273.0,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,NaN,NaN,NaN,4320031.0
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,19452021222,NaN,4,F,NaN,NaN,NaN,10621711.0,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,NaN,NaN,NaN,3540404.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,60292021222,NaN,2,F,NaN,NaN,NaN,NaN,4176523.0
10285,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,1322019222,NaN,6,F,NaN,NaN,NaN,NaN,NaN
10286,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,69152021222,NaN,5,F,NaN,NaN,NaN,NaN,NaN
10287,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2,60292019122,NaN,5,F,NaN,NaN,NaN,NaN,NaN


# Pegando valores das colunas cod_prod_pai_gen e cod_prod_mae_gen e atualizando colunas  cod_pai e cod_mae no DF_DADOS_NOVOS

In [17]:
df_temporaria6 = df_temporaria5
df_dados_novos_para_insercao_no_banco_de_dados['cod_pai'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_pai_gen'], inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_mae_gen'], inplace=True)
#Dando drop nas colunas antigas
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_pai_gen')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_mae_gen')
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,7461,2,2,74612019122,NaN,5,F,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,10639,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,6039,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,1945,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,73119,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10243,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,6029,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN
10285,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,132,2,2,1322019222,NaN,6,F,NaN,NaN,NaN
10286,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,6915,2,2,69152021222,NaN,5,F,NaN,NaN,NaN
10287,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,6029,2,2,60292019122,NaN,5,F,NaN,NaN,NaN


# Criando subtabela em que o gs do pai = 11 mas os 3 primeiros caracteres do rgpai nao sao letras

In [18]:
sub_tabela_cod_pai_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSPAI'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_pai_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[~sub_tabela_cod_pai_0['nomea'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_pai no genealogia_abcz dos animais em que o registro nao comeca com 3 letras

In [19]:
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
df_genealogia_abcz = df_genealogia_abcz.drop_duplicates()
#criando coluna com os 10 caracteres do nome
sub_tabela_cod_pai_0['nome11caracteres'] = sub_tabela_cod_pai_0['nomea'].str.slice(start=0, stop=11)
df_genealogia_abcz['nome11caracteres'] = df_genealogia_abcz['nome'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
#sub_tabela_cod_pai_0['nome10caracteres'] = sub_tabela_cod_pai_0['nome10caracteres'].str.replace('\s+', '', regex=True)
#df_genealogia_abcz['nome10caracteres'] = df_genealogia_abcz['nome10caracteres'].str.replace('\s+', '', regex=True)

sub_tabela_cod_pai_0 = pd.merge(sub_tabela_cod_pai_0, df_genealogia_abcz[['RGVACA', 'nome11caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome11caracteres'], right_on=['RGVACA', 'nome11caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop_duplicates()
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0.drop(sub_tabela_cod_pai_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_pai_0 = sub_tabela_cod_pai_0[sub_tabela_cod_pai_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_pai_0

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,nome11caracteres,cod_prod_pai_gen


# Adicionando o cod_prod_pai_gen dos pais no dataframe df_dados_novos
obs: dos REGPAI que nao comecam com 3 caracteres

In [20]:
df_temporaria7 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_pai_0[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria7 = df_temporaria7.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria7
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_pai_gen
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,NaN
1035,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,NaN,NaN,NaN
1036,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,NaN,NaN,NaN
1037,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2,2,60292019122,NaN,5,F,NaN,NaN,NaN,NaN


# Criando subtabela em que o gs da mae = 11 mas os 3 primeiros caracteres do rgmae nao sao letras

In [21]:
sub_tabela_cod_mae_0 = df_dados_novos_para_insercao_no_banco_de_dados[df_dados_novos_para_insercao_no_banco_de_dados['GSMAE'] == 11]
#e o cod_mae e nulo
sub_tabela_cod_mae_0
# selecionar apenas as linhas que NÃO começam com letras
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[~sub_tabela_cod_mae_0['nomea'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

# Pegando cod_prod_mae no genealogia abcz dos animais em que o registro nao comeca com 3 letras

In [22]:
sub_tabela_cod_mae_0['nome11caracteres'] = sub_tabela_cod_mae_0['nomea'].str.slice(start=0, stop=11)
df_genealogia_abcz = df_genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
sub_tabela_cod_mae_0 = pd.merge(sub_tabela_cod_mae_0, df_genealogia_abcz[['RGVACA','nome11caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome11caracteres'], right_on=['RGVACA', 'nome11caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop_duplicates()
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0.drop(sub_tabela_cod_mae_0.filter(regex='_drop$').columns, axis=1)
sub_tabela_cod_mae_0 = sub_tabela_cod_mae_0[sub_tabela_cod_mae_0['_merge'] == 'both'].drop(columns='_merge')
sub_tabela_cod_mae_0


,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_pai_gen,nome11caracteres,cod_prod_mae_gen


# Adicionando cod_prod_mae_gen no df_dados_novos
Obs: regmae com menos de 3 caracteres

In [23]:
df_temporaria8 = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, sub_tabela_cod_mae_0[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
#df_dados_novos_para_insercao_no_banco_de_dados=df_temporaria5
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()
#df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(df_dados_novos_para_insercao_no_banco_de_dados.filter(regex='_drop$').columns, axis=1)
#df_dados_novos_para_insercao_no_banco_de_dados
df_temporaria8 = df_temporaria8.drop_duplicates()
df_dados_novos_para_insercao_no_banco_de_dados = df_temporaria8
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,cod_prod_pai_gen,cod_prod_mae_gen
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,74612019122,NaN,5,F,NaN,NaN,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,NaN,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,NaN,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,NaN,NaN
1035,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,1322019222,NaN,6,F,NaN,NaN,NaN,NaN,NaN
1036,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,69152021222,NaN,5,F,NaN,NaN,NaN,NaN,NaN
1037,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,2,60292019122,NaN,5,F,NaN,NaN,NaN,NaN,NaN


# Adicionando os dados do cod_prod_mae_gen e do cod_prod_pai_gen no cod_pai e cod_mae
OBS: dados com o regpai e regmae com < 3 letras

In [24]:
df_dados_novos_para_insercao_no_banco_de_dados['cod_pai'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_pai_gen'], inplace=True)
df_dados_novos_para_insercao_no_banco_de_dados['cod_mae'].fillna(df_dados_novos_para_insercao_no_banco_de_dados['cod_prod_mae_gen'], inplace=True)
#Dando drop nas colunas antigas
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_pai_gen')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop(columns='cod_prod_mae_gen')
df_dados_novos_para_insercao_no_banco_de_dados

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rebc,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,7461,2,2,74612019122,NaN,5,F,NaN,NaN,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,10639,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,6039,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,1945,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,73119,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,6029,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN
1035,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,132,2,2,1322019222,NaN,6,F,NaN,NaN,NaN
1036,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,6915,2,2,69152021222,NaN,5,F,NaN,NaN,NaN
1037,6029,GERMANA FEBO FR RECREIO,5858,9069-AS,FEBO FR RECREIO,58,6300-N,CAMPANHA FR RECREIO,58,6180-N,...,6029,2,2,60292019122,NaN,5,F,NaN,NaN,NaN


# Adicionando reb no dataframe df_dados_novos_para_insercao_no_banco_de_dados
Obs: Cod_gil esta na tabela de criadores

In [25]:
df_criadores = df_criadores.rename(columns={'codgiro':'reb_giro'})

df_dados_novos_para_insercao_no_banco_de_dados = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, df_criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
df_dados_novos_para_insercao_no_banco_de_dados = df_dados_novos_para_insercao_no_banco_de_dados.drop_duplicates()


In [26]:
#Pegando os dados das colunas

In [27]:
#verifica_reb = df_criadores.groupby('reb_giro')['reb'].nunique()
#verifica_reb

## Calculando a diferença entre a data dos novos partos com as ultimas datas presentes na tabela de dados_2022 para validar se ha alguma inconsistência

obs: Caso a diferença seja menor que 250 dias é provável que haja alguma falha.

In [28]:

df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
#df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

,parto,parto_drop,diferenca_em_dias
0,2021-11-23,2020-01-09,684
1,2020-10-09,2019-01-31,617
2,2021-03-15,2020-01-24,416
3,2021-03-15,2018-02-01,1138
4,2021-06-28,2020-07-22,341
...,...,...,...
682,2021-03-23,2017-11-17,1222
683,2021-03-23,2020-03-10,378
684,2021-01-25,2019-05-03,633
685,2021-04-22,2020-05-13,344


## Pegando da tabela de animais que serão inseridos no banco aqueles que ja possuem IDANIMAL cadastrado
Comparado ao dataframe  dados_2022

In [29]:
df_animais_com_ID = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on =['IDANIMAL'], suffixes=('', '_drop'))
df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,2,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,3,2,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,2,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO
4,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2,2,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO
5,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2,2,55692021222,NaN,3,F,NaN,NaN,NaN,5569-GIRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,1129,MORADIA ANGICO,7,9556-F,VALE OURO DE BRASILIA,11,A6796,CABANA ANGICO,78,1606,...,2,2,11292021222,NaN,4,F,NaN,NaN,NaN,52721
681,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2,2,23812020122,NaN,2,F,NaN,3287666.0,NaN,31300-ABCZ
684,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,2,2,55692020122,NaN,2,F,NaN,3890526.0,NaN,5569-GIRO
685,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ


## Pegando da tabela de animais que serão inseridos no banco, todos aqueles que não possuem IDANIMAL cadastrado
comparado ao dataframe dados_2022

In [30]:
df_animais_sem_id = pd.merge(df_dados_novos_para_insercao_no_banco_de_dados, dados_2022, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,NaN,NaN,1291
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,2,19452022222,NaN,4,F,NaN,NaN,11428022.0,37634-ABCZ
1302,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,2,23812021222,NaN,4,F,NaN,NaN,12866148.0,31300-ABCZ
1305,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ
1306,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ


In [31]:
#dados_2022[dados_2022['IDANIMAL'] == 1631099]
#df_verificando_animais_com_id_na_genealogia_embrapa['IDANIMAL'].value_counts()
#verifica_reb = df_verificando_animais_com_id_na_genealogia_embrapa.groupby('IDANIMAL')['id_produto'].nunique()
#verifica_reb

# Adicionando a coluna id_produto ao dataframe df_animais_com_ID

In [ ]:
df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
df_animais_com_ID = pd.merge(df_animais_com_ID, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
df_animais_com_ID = df_animais_com_ID.drop_duplicates()
df_animais_com_ID = df_animais_com_ID.drop(columns='_merge')
df_animais_com_ID

## Lista de animais do dataframe sem ID em relacao a dados_2022 e que possuem ID_animal no dataframe da genealogia_Embrapa
Obs: Tem que ter ID_produto

In [ ]:
df_animais_sem_id = df_animais_sem_id.add_suffix('_esq')
df_genealogia_embrapa = df_genealogia_embrapa.add_suffix('_dir')

df_animais_sem_id_2022_que_possuem_id_no_banco = pd.merge(df_animais_sem_id, df_genealogia_embrapa, left_on=['IDANIMAL_esq'], right_on=['IDANIMAL_dir'], how='left', indicator=True)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop_duplicates()
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco[df_animais_sem_id_2022_que_possuem_id_no_banco['_merge'] == 'both']

# dropa as colunas que não são necessárias
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(df_animais_sem_id_2022_que_possuem_id_no_banco.filter(regex='_dir').columns.difference(['id_produto_dir']), axis=1)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco = df_animais_sem_id_2022_que_possuem_id_no_banco.drop(columns='_merge')
df_animais_sem_id_2022_que_possuem_id_no_banco

# Atualizando lista de animais que possuem ID encontrado

In [ ]:
# Juntando os dataframes
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_sem_id_2022_que_possuem_id_no_banco])
df_animais_com_ID

# Atualizando lista de animais sem ID

In [ ]:
#Tirando o sufixo da tabela
df_animais_sem_id = df_animais_sem_id.rename(columns=lambda x: x.replace('_esq', '') if '_esq' in x else x)
# Imprimir o resultado
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto'].isnull()]
df_animais_sem_id

# Fazendo busca no dataframe producao para encontrar id_produto dos animais sem id
conferir manualmente se entram ou não no banco

In [ ]:
#Criando coluna com o ano de parto e mes parto no dataframe df_producao
df_producao['parto'] = df_producao['parto'].astype(str)
df_producao['AP'] = df_producao['parto'].str.slice(start=0, stop=4)
df_producao['MP'] = df_producao['parto'].str.slice(start=5, stop=7)
#Pegando os 10 primeiros caracteres de nome dos dataframes aniamis sem id e producao
df_producao['nome10caracteres'] = df_producao['nome'].str.slice(start=0, stop=11)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nomea'].str.slice(start=0, stop=11)
#removendo os espacos em branco das celulas
df_producao['nome10caracteres'] = df_producao['nome10caracteres'].str.replace('\s+', '', regex=True)
df_animais_sem_id['nome10caracteres'] = df_animais_sem_id['nome10caracteres'].str.replace('\s+', '', regex=True)

df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_producao = df_producao.drop(df_producao.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns='_merge')

df_producao['AP'] = df_producao['AP'].astype(int)
df_producao['MP'] = df_producao['MP'].astype(int)
df_animais_sem_id['AP'] = df_animais_sem_id['AP'].astype(int)
df_animais_sem_id['MP'] = df_animais_sem_id['MP'].astype(int)
df_animais_com_id_na_producao = pd.merge(df_animais_sem_id, df_producao[['reb','AP', 'MP', 'nome10caracteres','id_produto']], on=['reb', 'AP', 'MP', 'nome10caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_producao = df_animais_com_id_na_producao[df_animais_com_id_na_producao['_merge'] == 'both']
df_animais_com_id_na_producao = df_animais_com_id_na_producao.drop(columns={'_merge', 'id_produto'})
df_animais_com_id_na_producao = df_animais_com_id_na_producao.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_producao

# Atualizando animais com id com os novos ids_produtos encontrados no dataframe de producao

In [ ]:
df_animais_com_ID = pd.concat([df_animais_com_ID, df_animais_com_id_na_producao])
df_animais_com_ID

# Atualizando lista de animais sem id

In [ ]:
df_animais_sem_id = pd.merge(df_animais_sem_id, df_animais_com_ID, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['id_produto_drop'].isnull()]
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
# Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"
df_animais_sem_id

# Fazendo a busca na tabela genealogia_embrapa
Fazer verificacao manual

In [ ]:
#Retirando sufixo tabela genealogia
df_genealogia_embrapa = df_genealogia_embrapa.rename(columns=lambda x: x.replace('_dir', '') if '_dir' in x else x)
#Pegando ano nascimento nas colunas
df_genealogia_embrapa['nasc_est'] = df_genealogia_embrapa['nasc_est'].astype(str)
df_genealogia_embrapa['AN'] = df_genealogia_embrapa['nasc_est'].str.slice(start=0, stop=4)
df_animais_sem_id['nasc'] = df_animais_sem_id['nasc'].astype(str)
df_animais_sem_id['AN'] = df_animais_sem_id['nasc'].str.slice(start=0, stop=4)
#Criando coluna com os 10 caracteres do nome na genealogia_embrapa
df_genealogia_embrapa['nome10caracteres'] = df_genealogia_embrapa['nome'].str.slice(start=0, stop=11)
#Convertendo coluna ano e mes para tipo inteiro
#df_genealogia_embrapa['AN'] = df_genealogia_embrapa['AN'].astype(int)
#Limpando tabelas
df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
df_genealogia_embrapa = df_genealogia_embrapa.drop(df_genealogia_embrapa.filter(regex='_drop$').columns, axis=1)
df_animais_sem_id = df_animais_sem_id.drop(columns={'_merge'})
#criando dataframe com os ids_encontrados
df_animais_com_id_na_genealogia = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs','RGMAE', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_mae', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia[df_animais_com_id_na_genealogia['_merge'] == 'both']
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(columns={'_merge', 'id_produto', 'registro_mae', 'registro_pai'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.rename(columns={'id_produto_drop': 'id_produto'})
df_animais_com_id_na_genealogia = df_animais_com_id_na_genealogia.drop(df_animais_com_id_na_genealogia.filter(regex='_drop$').columns, axis=1)
df_animais_com_id_na_genealogia

In [ ]:
#Conferência manual dos matchs com o dataframe da genealogia

In [ ]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'RGPAI', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'registro_pai', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

# Adicionar o que ta escrito no quadro

In [ ]:
df_animais_com_id_na_genealogia1 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['nome10caracteres','gs', 'sexo', 'AN','id_produto']], left_on=['nome10caracteres', 'gs', 'sexo', 'AN'], right_on=['nome10caracteres','gs', 'sexo', 'AN'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia1 = df_animais_com_id_na_genealogia1[df_animais_com_id_na_genealogia1['_merge'] == 'both']
df_animais_com_id_na_genealogia1

In [ ]:
df_animais_com_id_na_genealogia2 = pd.merge(df_animais_sem_id, df_genealogia_embrapa[['id_produto','cod_prod']], left_on=['cod_prod'], right_on=['cod_prod'], how='left', indicator=True, suffixes=('', '_drop'))
df_animais_com_id_na_genealogia2 = df_animais_com_id_na_genealogia2[df_animais_com_id_na_genealogia2['_merge'] == 'both']
df_animais_com_id_na_genealogia2